# 배우의 등급 평가

In [49]:
import numpy as np
import pandas as pd

In [50]:
actor_brand_d = pd.read_csv(r"C:\Users\sjung\Desktop\Data_Science_Lab\EDA_project\10th-EDA\Team_C\정제데이터\배우브랜드.csv")

In [51]:
actor_names = actor_brand_d['브랜드'].unique()
periods = actor_brand_d['기간'].unique()

# 1. Z score 활용
각 배우별로 해당 기간에서 상대적인 순위를 알아보자
브랜드평판지수가 배우의 종합적인 브랜드 가치를 나타내는 지표이므로 해당 지표의 z score를 기준으로 배우의 실질적인 순위를 찾아봤음.

In [4]:
actor_brand_d_z = actor_brand_d.copy()
actor_brand_d_z['z_score'] = np.nan # z score 열을 추가
actor_brand_d_z.head()

,기간,순위,브랜드,참여지수,미디어지수,소통지수,커뮤니티지수,브랜드평판지수,z_score
0,1901,1,염정아,1372024,3513112,2794528,2282605,9962269,NaN
1,1901,2,현빈,2894066,2788296,1695680,2233463,9611504,NaN
2,1901,3,송혜교,1746374,2978220,3207168,1676911,9608673,NaN
3,1901,4,박보검,1933351,2855088,2242528,2345178,9376145,NaN
4,1901,5,유이,831162,3082884,3204288,1530746,8649080,NaN


In [5]:
# z score를 기준으로 상대적인 점수를 저장할 열
actor_brand_d_z['relative_score'] = np.nan

for period in list(periods):
    period_data = actor_brand_d_z.loc[actor_brand_d_z['기간']==period,:].copy()
    p_data = period_data['브랜드평판지수'].copy()
    actor_brand_d_z.loc[actor_brand_d_z['기간']==period,'z_score'] =\
                            (p_data - np.mean(p_data)) / np.std(p_data)


## 배우의 z_score를 기준으로 점수 부여<br>
(* z score가 -2 이하인 배우는 없었음)
(* z score 5점대와 6점대는 박은빈 배우밖에 없음)
| z score            | 점수 |
|--------------------|------|
| 5 <= z_score       | 31   |
| 4 <= z_score < 5   | 25   |
| 3 <= z_score < 4   | 20   |
| 2 <= z_score < 3   | 16   |
| 1 <= z_score < 2   | 13   |
| 0 <= z_score < 1   | 11   |
| -1 <= z_score < 0  | 10   |
| -2 <= z_score < -1 | 8    |
| -3 <= z_score < -2 | 5    |
|       z_score < -3| 1    |



In [6]:
actor_brand_d_z.loc[5<=actor_brand_d_z['z_score'], 'relative_score'] = 31
actor_brand_d_z.loc[(4<=actor_brand_d_z['z_score'])&(actor_brand_d_z['z_score']<5),
                                                    'relative_score'] = 25
actor_brand_d_z.loc[(3<=actor_brand_d_z['z_score'])&(actor_brand_d_z['z_score']<4),
                                                    'relative_score'] = 20
actor_brand_d_z.loc[(2<=actor_brand_d_z['z_score'])&(actor_brand_d_z['z_score']<3),
                                                    'relative_score'] = 16
actor_brand_d_z.loc[(1<=actor_brand_d_z['z_score'])&(actor_brand_d_z['z_score']<2),
                                                    'relative_score'] = 13
actor_brand_d_z.loc[(0<=actor_brand_d_z['z_score'])&(actor_brand_d_z['z_score']<1),
                                                    'relative_score'] = 11
actor_brand_d_z.loc[(-1<=actor_brand_d_z['z_score'])&(actor_brand_d_z['z_score']<0),
                                                    'relative_score'] = 10
actor_brand_d_z.loc[(-2<=actor_brand_d_z['z_score'])&(actor_brand_d_z['z_score']<-1),
                                                    'relative_score'] = 8
actor_brand_d_z.loc[(-3<=actor_brand_d_z['z_score'])&(actor_brand_d_z['z_score']<-2),
                                                    'relative_score'] = 5
actor_brand_d_z.loc[(actor_brand_d_z['z_score']<-3), 'relative_score'] = 1

## 배우별 등급 산출
배우별로 relative score의 합을 구한 뒤,<br>
각 배우의 z score를 다시 산출해 해당 값을 기준으로 배우의 등급 나눔


In [7]:
# 배우별 relative score의 합
actor_score = actor_brand_d_z.groupby('브랜드').agg(['sum'])['relative_score']
actor_score['z_score'] = np.nan
score_data = actor_score['sum']
actor_score['z_score'] = (score_data - np.mean(score_data)) / np.std(score_data)
actor_score

,sum,z_score
브랜드,,
감우성,10.0,-0.872382
강기둥,10.0,-0.872382
강기영,22.0,-0.230813
강말금,20.0,-0.337741
강민아,20.0,-0.337741
...,...,...
황승언,28.0,0.089972
황신혜,62.0,1.907753
황인엽,21.0,-0.284277


전체 기간 기준, z score가 -1 미만인 배우는 없었음(전체 배우 수: 618명)
| z score            | 숫자  | 등급 |
|--------------------|------|------|
| 3 <= z_score       | 9    |S|
| 2 <= z_score < 3   | 23   |A|
| 1 <= z_score < 2   | 57   |B|
| 0 <= z_score < 1   | 151  |C|
| -1 <= z_score < 0  | 378  |D|


In [8]:
actor_score['grade'] = np.nan
actor_score.loc[3 <= actor_score['z_score'], 'grade'] = 'S'
actor_score.loc[(2<=actor_score['z_score']) & (actor_score['z_score']<3), 'grade'] = 'A'
actor_score.loc[(1<=actor_score['z_score']) & (actor_score['z_score']<2), 'grade'] = 'B'
actor_score.loc[(0<=actor_score['z_score']) & (actor_score['z_score']<1), 'grade'] = 'C'
actor_score.loc[(actor_score['z_score']<0), 'grade'] = 'D'

## 최종 결과

In [9]:
actor_score

,sum,z_score,grade
브랜드,,,
감우성,10.0,-0.872382,D
강기둥,10.0,-0.872382,D
강기영,22.0,-0.230813,D
강말금,20.0,-0.337741,D
강민아,20.0,-0.337741,D
...,...,...,...
황승언,28.0,0.089972,C
황신혜,62.0,1.907753,B
황인엽,21.0,-0.284277,D


## 기간에 따라 배우별 등급 산출하는 함수 구현

In [109]:
# z score 열을 추가하는 함수
def add_z_score(actor_brand_d):
    actor_brand_d_z = actor_brand_d.copy()
    actor_brand_d_z['z_score'] = np.nan # z score 열을 추가
    return actor_brand_d_z

def add_relative_score(actor_brand_d_z):
    # z score를 기준으로 상대적인 점수를 저장할 열
    actor_brand_d_z['relative_score'] = np.nan
    periods = actor_brand_d['기간'].unique()
    for period in list(periods):
        period_data = actor_brand_d_z.loc[actor_brand_d_z['기간']==period,:].copy()
        p_data = period_data['브랜드평판지수'].copy()
        actor_brand_d_z.loc[actor_brand_d_z['기간']==period,'z_score'] =\
                                (p_data - np.mean(p_data)) / np.std(p_data)
    return actor_brand_d_z

def calculate_relative_score(actor_brand_d_z):
    actor_brand_d_z.loc[5<=actor_brand_d_z['z_score'], 'relative_score'] = 31
    actor_brand_d_z.loc[(4<=actor_brand_d_z['z_score'])&(actor_brand_d_z['z_score']<5),
                                                        'relative_score'] = 25
    actor_brand_d_z.loc[(3<=actor_brand_d_z['z_score'])&(actor_brand_d_z['z_score']<4),
                                                        'relative_score'] = 20
    actor_brand_d_z.loc[(2<=actor_brand_d_z['z_score'])&(actor_brand_d_z['z_score']<3),
                                                        'relative_score'] = 16
    actor_brand_d_z.loc[(1<=actor_brand_d_z['z_score'])&(actor_brand_d_z['z_score']<2),
                                                        'relative_score'] = 13
    actor_brand_d_z.loc[(0<=actor_brand_d_z['z_score'])&(actor_brand_d_z['z_score']<1),
                                                        'relative_score'] = 11
    actor_brand_d_z.loc[(-1<=actor_brand_d_z['z_score'])&(actor_brand_d_z['z_score']<0),
                                                        'relative_score'] = 10
    actor_brand_d_z.loc[(-2<=actor_brand_d_z['z_score'])&(actor_brand_d_z['z_score']<-1),
                                                        'relative_score'] = 8
    actor_brand_d_z.loc[(-3<=actor_brand_d_z['z_score'])&(actor_brand_d_z['z_score']<-2),
                                                        'relative_score'] = 5
    actor_brand_d_z.loc[(actor_brand_d_z['z_score']<-3), 'relative_score'] = 1
    return actor_brand_d_z

def rate_actors(actor_brand_d_z): # 배우별 relative score의 총합을 기준으로 배우의 등급 산출
    # 배우별 relative score의 합
    actor_score = actor_brand_d_z.groupby('브랜드').agg(['sum'])['relative_score']
    actor_score['z_score'] = np.nan
    score_data = actor_score['sum']
    actor_score['z_score'] = (score_data - np.mean(score_data)) / np.std(score_data)
    actor_score['grade'] = np.nan
    # 배우의 등급산출
    actor_score.loc[3 <= actor_score['z_score'], 'grade'] = 'S'
    actor_score.loc[(2<=actor_score['z_score']) & (actor_score['z_score']<3), 'grade'] = 'A'
    actor_score.loc[(1<=actor_score['z_score']) & (actor_score['z_score']<2), 'grade'] = 'B'
    actor_score.loc[(0<=actor_score['z_score']) & (actor_score['z_score']<1), 'grade'] = 'C'
    actor_score.loc[(actor_score['z_score']<0), 'grade'] = 'D'
    return actor_score

In [110]:
# 최종적으로 원하는 기간을 입력하면 배우별 등급을 산출해주는 함수
# starting_point: 시작 기간. int형식. 연월을 입력 ex. 1902, 2206
# ending_point: 끝 기간. int형식. 연월을 입력 ex. 1902, 2206
def get_acotr_grade(actor_brand_d, starting_point = 1901, ending_point = 2306):
    actor_brand_d = actor_brand_d[(starting_point<=actor_brand_d['기간']) &\
                                                (actor_brand_d['기간']<=ending_point)]
    actor_brand_d.reset_index(inplace = True, drop=True)
    actor_brand_d_z = add_z_score(actor_brand_d)
    actor_brand_d_z = add_relative_score(actor_brand_d_z)
    actor_brand_d_z = calculate_relative_score(actor_brand_d_z)
    actor_score = rate_actors(actor_brand_d_z)
    actor_score.reset_index(inplace = True)
    actor_score.drop(labels=['sum','z_score'], axis=1, inplace=True)
    return actor_score

### 함수 실행
- starting_point: 시작 기간(1901 ~ 2306)
- ending_point: 끝 기간(1901 ~ 2306)
- starting_point <= ending_point이어야 함
- starting_point == ending_point이면 해당 월을 기준으로한 배우 등급이 나옴

In [111]:
starting_point = 1901
ending_point = 2306
grade_d = get_acotr_grade(actor_brand_d, starting_point, ending_point)

grade_d['grade'].value_counts().sort_values() # 등급별 인원이 얼마나 있는지 확인

S      9
A     23
B     57
C    151
D    378
Name: grade, dtype: int64

In [112]:
grade_d

,브랜드,grade
0,감우성,D
1,강기둥,D
2,강기영,D
3,강말금,D
4,강민아,D
...,...,...
613,황승언,C
614,황신혜,B
615,황인엽,D
616,황정음,D


## 결과 저장

In [116]:
saving_filename = 'actor_ranking_(z_score_based)_'+str(starting_point) + '_' + str(ending_point) + '.csv'
saving_path = 'c:\\Users\\sjung\\Desktop\\Data_Science_Lab\\EDA_project\\10th-EDA\\Team_C\\정제데이터\\'
grade_d.to_csv(saving_path+saving_filename, index = False, encoding='utf-8-sig')